<a href="https://colab.research.google.com/github/JayVeer18/CUDA-Programs/blob/main/Dot_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step:1

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

### Step:2

In [ ]:

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

### Step:3

In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


### Step-4

In [ ]:
pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

### Step-5

In [12]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


### Dot Product using Shared Memory

In [13]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define imin(a,b) (a<b?a:b)

const int N = 33*1024;
const int threadsperblock = 256;
const int blockspergrid = imin(32, (N+threadsperblock-1)/threadsperblock);
__global__ void dot(int *a, int *b, int *c){
    __shared__ int cache[threadsperblock];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cacheindex = threadIdx.x;
    int temp = 0;
    while(tid < N){
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x; 
    } 
    cache[cacheindex] = temp;
    __syncthreads();
    int i = blockDim.x/2;
    while(i != 0){
        if(cacheindex < i){
            cache[cacheindex] += cache[cacheindex + i];
            __syncthreads();
        }
        i /= 2;
    }
    if(cacheindex == 0){
        c[blockIdx.x] = cache[0];
    }
}
int main()
{
  int *a,*b,*partial_c;
  int *da, *db, *dc;
  int size = N*sizeof(int);
  a = new int[N];
  b = new int[N];
  //partial_c = (int*)malloc(blockspergrid*sizeof(int));
  partial_c = new int[blockspergrid];
 
  cudaMalloc((void **)&da, size);
  cudaMalloc((void **)&db, size);
  cudaMalloc((void **)&dc, blockspergrid*sizeof(int));
 int res=0;
 for(int i=0;i<N;i++){
      a[i] = i;
      b[i] = i * 2;
      res += (a[i] * b[i]);
  }
 
  cudaMemcpy(da, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(db, b, size, cudaMemcpyHostToDevice);
  
  dot<<<blockspergrid,threadsperblock>>>(da, db, dc);
  
  cudaMemcpy(partial_c, dc, blockspergrid*sizeof(int), cudaMemcpyDeviceToHost);
  
  int c=0;
  for(int i=0;i<blockspergrid;i++){
      c+=partial_c[i];
  }
  printf("Dot Product from GPU : %d \n Dot Product from CPU : %d",c,res);

  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);
  return 0;
}

Dot Product from GPU : 1005595648 
 Dot Product from CPU : 1005595648


### Dot Product using Atomics


In [14]:
%%cu
#include <stdio.h>
#include <stdlib.h>

const int N = 500;
__global__ void dotProduct(int *a, int *b, int *c)
{
  int i = threadIdx.x;
  atomicAdd(c, a[i]*b[i]);
 
}
int main()
{
  int *a,*b;
  int *da, *db, *dc;
  int size = N*sizeof(int);
  a = new int[N];
  b = new int[N];
  int res=0,c=0;
  for(int i=0;i<N;i++){
        a[i] = i;
        b[i] = i * 2;
        res += (a[i] * b[i]);
    }
  cudaMalloc((void **)&da, size);
  cudaMalloc((void **)&db, size);
  cudaMalloc((void **)&dc, sizeof(int));
 
  cudaMemcpy(da, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(db, b, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dc, &c, sizeof(int), cudaMemcpyHostToDevice);

  dotProduct<<<1,N>>>(da, db, dc);
 
  cudaMemcpy(&c, dc, sizeof(int), cudaMemcpyDeviceToHost);
  printf("Dot Product from GPU : %d \n Dot Product from CPU : %d",c,res);

  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);
  return 0;
}

Dot Product from GPU : 83083500 
 Dot Product from CPU : 83083500


***Dot Product using Atomics with Shared Memory***

In [30]:
%%cu 
#include<stdio.h>
using namespace std;
#define imin(a,b) (a<b?a:b)

const int N = 10;
const int threadsperblock = 5;
const int numberOfBlocks = 2;

__global__ void dotProduct(float *a,float *b,float *c,float *d){
    int threadid = threadIdx.x + blockIdx.x * blockDim.x;
    int blockid = blockIdx.x;
    __shared__ float sm[numberOfBlocks];
    d[threadid] += a[threadid] * b[threadid];
    atomicAdd(&sm[blockid],d[threadid]);
    c[blockid] = sm[blockid];
    __syncthreads();
}
int main(){
    float *a,*b,*c,*d;
    float *dev_a,*dev_b,*dev_c,*dev_d;
    int numberOfBytes = N*sizeof(float);
    a = (float*)malloc(numberOfBytes);
    b = (float*)malloc(numberOfBytes);
    c = (float*)malloc(numberOfBytes);
    d = (float*)malloc(numberOfBytes);
    for(int i=0;i<N;i++){
      a[i] = i;
      b[i] = i * 2;
  }
  cudaMalloc((void**)&dev_a,numberOfBytes);
  cudaMalloc((void**)&dev_b,numberOfBytes);
  cudaMalloc((void**)&dev_c,numberOfBlocks*sizeof(float));
  cudaMalloc((void**)&dev_d,numberOfBytes);

  cudaMemcpy(dev_a,a,numberOfBytes,cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b,b,numberOfBytes,cudaMemcpyHostToDevice);
 
  cudaMemset(dev_c,0,numberOfBlocks*sizeof(float));
  cudaMemset(dev_d,0,numberOfBytes);

  cudaEvent_t start,stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaEventRecord(start,0);

  dotProduct<<<numberOfBlocks,threadsperblock>>>(dev_a,dev_b,dev_c,dev_d);

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  float time = 0;
  cudaEventElapsedTime(&time,start,stop);

  cudaMemcpy(c,dev_c,numberOfBlocks*sizeof(float),cudaMemcpyDeviceToHost);
  cudaMemcpy(d,dev_d,numberOfBytes,cudaMemcpyDeviceToHost);
  printf("Time taken: %f",time);

  float temp = 0;
  printf("\na[i]\t\tb[i]\t\tproduct\n");
  for(int i=0;i<N;i++){
      printf("%f\t%f\t%f\n",a[i],b[i],d[i]);
  }
  for(int i=0;i<numberOfBlocks;i++){
      temp += c[i];
  }
  printf("\n DotProduct...%f\n",temp);

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);
  cudaFree(dev_d);

  return 0;
}

Time taken: 0.034848
a[i]		b[i]		product
0.000000	0.000000	0.000000
1.000000	2.000000	2.000000
2.000000	4.000000	8.000000
3.000000	6.000000	18.000000
4.000000	8.000000	32.000000
5.000000	10.000000	50.000000
6.000000	12.000000	72.000000
7.000000	14.000000	98.000000
8.000000	16.000000	128.000000
9.000000	18.000000	162.000000

 DotProduct...570.000000

